In [1]:
from astropy.io import fits 
import matplotlib.pyplot as plt
import numpy as np 
from scipy.optimize import minimize, rosen, rosen_der
from scipy import interpolate
import scipy
import os 
from scipy.optimize import curve_fit
from scipy.integrate import simps
import math

In [2]:
#importing data file 
stuff = fits.open("C:/Users/19133/Documents/Research/Kirkpatrick/S82X_catalog_w_mbh_Cooke_Ricci.fits")

In [3]:
stuff[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  646 / length of dimension 1                          
NAXIS2  =                 6181 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                  119 / number of table fields                         
TTYPE1  = 'MSID    '                                                            
TFORM1  = 'J       '                                                            
TTYPE2  = 'REC_NO  '                                                            
TFORM2  = 'J       '                                                            
TTYPE3  = 'OBSID   '        

In [4]:
#MAG to FLUX 

#function for taking the magnitudes from the file and turning them into mJy 
#this does it for each band's list of values at once 
def mag2flux(m):  
    
    #making empty list to fill with corrected values 
    flux = np.zeros(len(m))
    
    for i in range(len(m)): 
        #if the magnitude is not detected, we replace it with a nan
        if m[i] <= -99: 
            flux[i] = np.nan 
            
        else: 
            #doing the conversion 
            power = m[i]/(-5/2)
            flux[i] = (10**power)*3631#*1000?
    return flux #returns list of fluxes 

In [5]:
#MAG to FLUX ERROR

#doing same as above but with the error calculation 
def mag2flux_err(m_err,flux): 
    
    flux = np.zeros(len(m_err))
    
    for i in range(len(m_err)): 
        if m_err[i] <= -99: 
            flux[i] = np.nan 
        
        else: 
            flux[i] = m_err[i]*flux[i]*np.log(10)
    return flux 

In [6]:
#XRAY to MJY 

def xray_to_mjy(m,kev):
    freq = kev*(2.41799*(10**(17)))
    l = np.zeros(len(m))
    for i in range(len(m)): 
        if m[i] == 0: 
            l[i] = np.nan 
        else: 
            mjy = m[i]*1000/(freq*10**(-23))
            l[i] = mjy
    return l 

In [7]:
#Function to correct wavelength for redshift 

def correct_redshift(z,wavelength): 
    return wavelength/(1+z)

In [8]:
#FLUX back to MAG 

def flux2mag(f):
    mag = (-2.5)*(np.log10(f/3631))
    return mag

In [9]:
#INTERPOLATION FUNCTION 

def interpolate(arr1,arr2,interparr): 
    from scipy import interpolate
    interp_vals = [] 
    #arr1 and arr2 are the x and y of the function that needs new values made 
    #interparr is the one with the corresponding x values where we need to interpolate a new point 
    for x in interparr: 
        leftx = arr1[arr1 < x].max() #closest point on left, x value
        rightx = arr1[arr1 > x].min() #closest point on right, x value
        lefty = arr2[np.where(arr1 == leftx)][0] #closest point on left, y value
        righty = arr2[np.where(arr1 == rightx)][0] #closest point on right, y value 
        interp = interpolate.interp1d([leftx,rightx],[lefty,righty]) #use the xs and ys to interpolate 
        val_for_match = interp(x) #use function from interp on wavelength value of galaxy
        interp_vals.append(val_for_match)
    return interp_vals 

In [10]:
#data has two id columns where it changes which one it used 
#so I'm just consolidating them into one list of unique ids for each source 
msids = stuff[1].data['MSID']
rec_nos = stuff[1].data['REC_NO']
msids[np.where(stuff[1].data['MSID'] == 0)] = rec_nos[np.where(stuff[1].data['MSID'] == 0)]
ids = msids

In [11]:
#Now we are just taking all the wavelengths from the file and converting them to the units we want 
FUV = mag2flux(stuff[1].data['MAG_FUV'])
FUV_err = mag2flux_err(stuff[1].data['MAGERR_FUV'],FUV)
NUV = mag2flux(stuff[1].data['MAG_NUV'])
NUV_err = mag2flux_err(stuff[1].data['MAGERR_NUV'],NUV)

#ugriz 
U = mag2flux(stuff[1].data['U'])
G = mag2flux(stuff[1].data['G'])
R = mag2flux(stuff[1].data['R'])
I = mag2flux(stuff[1].data['I'])
Z = mag2flux(stuff[1].data['Z'])

U_err = mag2flux_err(stuff[1].data['U_ERR'],U)
G_err = mag2flux_err(stuff[1].data['G_ERR'],G)
R_err = mag2flux_err(stuff[1].data['R_ERR'],R)
I_err = mag2flux_err(stuff[1].data['I_ERR'],I)
Z_err = mag2flux_err(stuff[1].data['Z_ERR'],Z)

#JHK
JVHS = mag2flux(stuff[1].data['JVHS'])
JVHS_err = mag2flux_err(stuff[1].data['JVHS_ERR'],JVHS)
HVHS = mag2flux(stuff[1].data['HVHS'])
HVHS_err = mag2flux_err(stuff[1].data['HVHS_ERR'],HVHS)
KVHS = mag2flux(stuff[1].data['KVHS'])
KVHS_err = mag2flux_err(stuff[1].data['KVHS_ERR'],KVHS)

JUK = mag2flux(stuff[1].data['JUK'])
JUK_err = mag2flux_err(stuff[1].data['JUK_ERR'],JUK)
HUK = mag2flux(stuff[1].data['HUK'])
HUK_err = mag2flux_err(stuff[1].data['HUK_ERR'],HUK)
KUK = mag2flux(stuff[1].data['KUK'])
KUK_err = mag2flux_err(stuff[1].data['KUK_ERR'],KUK)

#IRAC1, IRAC2
CH1 = mag2flux(stuff[1].data['CH1_SPIES'])
CH1_err = mag2flux_err(stuff[1].data['CH1_SPIES_ERR'],CH1)
CH2 = mag2flux(stuff[1].data['CH2_SPIES'])
CH2_err = mag2flux_err(stuff[1].data['CH2_SPIES_ERR'],CH2)

#W1, W2, W3, W4
W1 = mag2flux(stuff[1].data['W1'])
W1_err = mag2flux_err(stuff[1].data['W1_ERR'],W1)
W2 = mag2flux(stuff[1].data['W2'])
W2_err = mag2flux_err(stuff[1].data['W2_ERR'],W2)
W3 = mag2flux(stuff[1].data['W3'])
W3_err = mag2flux_err(stuff[1].data['W3_ERR'],W3)
W4 = mag2flux(stuff[1].data['W4'])
W4_err = mag2flux_err(stuff[1].data['W4_ERR'],W4)

X_hard = xray_to_mjy(stuff[1].data['HARD_FLUX'],5)
X_hard_err = xray_to_mjy(stuff[1].data['HARD_FLUX_ERROR'],5)
X_soft = xray_to_mjy(stuff[1].data['SOFT_FLUX'],1)
X_soft_err = xray_to_mjy(stuff[1].data['SOFT_FLUX_ERROR'],1)

#corresponding wavelength values to the bands in angstroms 
Jang = 12000
Hang = 16000
Kang = 22000

Uang = 3543
Gang = 4770
Rang = 6231
Iang = 7625
Zang = 9134

Vang = 5510

CH1ang = 35510
CH2ang = 44930

FUVang = 1550
NUVang = 2275

WISE1ang = 34000
WISE2ang = 46000
WISE3ang = 120000
WISE4ang = 220000

#importing templates for interpolation later 
Uf = np.genfromtxt("C:/Users/19133/Documents/Research/Kirkpatrick/Bessel_U.txt",invalid_raise = False)
Vf = np.genfromtxt("C:/Users/19133/Documents/Research/Kirkpatrick/Bessel_V.txt",invalid_raise = False)
Jf = np.genfromtxt("C:/Users/19133/Documents/Research/Kirkpatrick/VISTA_J.dat",invalid_raise = False)

#units of wavelengths in these files are in nm
#so change to angstroms 
Uf[:,0] = Uf[:,0]*10
Vf[:,0] = Vf[:,0]*10
Jf[:,0] = Jf[:,0]*10

C:\Users\19133\Anaconda3\lib\site-packages\ipykernel_launcher.py:82: ConversionWarning: Some errors were detected !
    Line #1802 (got 1 columns instead of 2)


In [12]:
#file comes with two redshift columns 
#spec is prefereable, so if we have it, use that
#otherwise use photo

spec_z = stuff[1].data['SPEC_Z']
photo_z = stuff[1].data['PHOTO_Z']

#they denote no detection in a couple ways, just turn them all to nans 
spec_z[np.where(spec_z <= -99)[0]] = np.nan
spec_z[np.where(spec_z == 0)[0]] = np.nan

#starting with spec
real_z = spec_z

#if spec has no detection, either use the photo or input nan if no photo as well 
for i in range(len(real_z)):
    if np.isnan(real_z[i]) == True: 
        if photo_z[i] <= -99:
            real_z[i] = np.nan
        else:
            real_z[i] = photo_z[i]
    else:
        True 

In [13]:
#making an array of the fluxes we want to use for the SED fitting 
fluxes = np.array([U,G,R,I,Z,JUK,HUK,KUK])

In [14]:
#removing sources with less than 2 bands detected because you can't fit with that 

#turning all nans to zeros so I can use np count nonzero function
fluxes[np.where(np.isnan(fluxes) == True)] = 0

indeces = [] 

#for each source, looking at its individual list of band fluxes 
#if there are less than two detections, save this index as one to get rid of 
for x in range(len(fluxes[0])):
    a = np.count_nonzero(fluxes[:,x])
    if a <= 1:
        indeces.append(x)
    #also taking away indeces that have galaxies without a redshift
    elif (np.isnan(real_z[x]) == True):
        indeces.append(x)
    else: 
        True 

#turning zeros back into nans to make math calculations work smoother later 
fluxes[np.where(fluxes == 0)] = np.nan

In [15]:
#removing aforementioned galaxies from the data I will deal with 
fluxes = np.delete(fluxes,indeces,1)
real_z = np.delete(real_z,indeces)
X_soft = np.delete(X_soft,indeces)
X_hard = np.delete(X_hard,indeces)
ids = np.delete(ids,indeces)

In [16]:
finaldata = open("C:/Users/19133/Documents/Research/Final_Data_4_6.txt", "w+")
finaldata.close()
bestfitdata = open("C:/Users/19133/Documents/Research/Bestfit_Data_4_6.txt", "w+")
bestfitdata.close()

In [17]:
#Defining functions to find paramters of 
#To fit template to points 

def f1(X,scale): 
    wavelength,tempflux = X
    return tempflux*scale

def f2(X,scale,alpha): 
    wavelength,tempflux = X
    return scale*tempflux**(alpha)

def f3(X,scale,tau): 
    wavelength,tempflux = X
    return scale*tempflux*(math.e)**(-1*tau)

In [ ]:
#use to look at individual errors 
i = 0
failed = [] 

#number of sources in data 
while i < 6100:
    try:

        #Source data  
        rshift = real_z[i]
        X_soft1 = X_soft[i]
        X_hard1 = X_hard[i]
        idd = ids[i]
        galaxy_fluxes = fluxes[:,i]
        
        #correcting wavelengths for source redshift 
        galaxy_wavelengths = np.array([Uang,Gang,Rang,Iang,Zang,Jang,Hang,Kang])
        for x in range(len(galaxy_wavelengths)):
            galaxy_wavelengths[x] = correct_redshift(rshift,galaxy_wavelengths[x])
        
        r2s = [] #Rsquared values from fitting 
        temps = [] #templates from fitting 
        popts = [] #parameters from fitting 
        fs = np.array([],dtype='object') #functions from fitting 
        
        #Fiting every SED I have to data 
        #picking out the best SED, fitting function, and parameters 
        
        #looping over each SED 
        for x in os.listdir("C:/Users/19133/Documents/Research/Kirkpatrick/Templates"):
            
            #choosing SED template 
            templatename = x
            template = np.genfromtxt("C:/Users/19133/Documents/Research/Kirkpatrick/Templates/" + x)

            #Template data 
            temp_x = template[:,0] #x axis of template is wavelength in angstroms (?)
            temp_y = template[:,1]*(temp_x**2)/(3*(10**8)) #changing l(lambda) to l(nu) 

            #this if statement skips SEDs where the template doesn't cover all the galaxy wavelengths 
            #at their corrected redshift values 
            
            if (min(galaxy_wavelengths) < min(temp_x)) == True or (max(galaxy_wavelengths) > max(temp_x)): 
                pass 
            
            #here we proceed 
            else: 
                #Getting flux values from template at correct x values for data (at each galaxy wavelength)
                #By interpolating 
                #This is needed to do the fitting 
            
                temp_fluxes = interpolate(temp_x,temp_y,galaxy_wavelengths)
                temp_fluxes = np.array([temp_fluxes])[0]

                #Taking places where the galaxy has detected photometry 
                #So we only do scaling calculations on those
                inds = np.where(np.isnan(galaxy_fluxes) == False)[0]
                #taking only data that was detected now 
                galaxy_fluxes = galaxy_fluxes[inds]
                temp_fluxes = temp_fluxes[inds]
                galaxy_wavelengths = galaxy_wavelengths[inds]
                
                #now we move on to, for this SED, trying each of the 3 available scaling functions
                #and trying to fit them with various parameters 
                
                try:
                #function 1 
                    popt, pcov = curve_fit(f1,(galaxy_wavelengths,temp_fluxes),galaxy_fluxes,p0=[10],maxfev=500000)
                    #Calculating the r**2 value for this template/scaling factor 
                    num = np.sum((galaxy_fluxes - f1((galaxy_wavelengths,temp_fluxes), *popt))**2)
                    denom = np.sum((galaxy_fluxes - np.mean(galaxy_fluxes))**2)
                    r2 = 1 - num/denom 
                except: 
                    failed.append(i)
                
                try:
                    #function 2 
                    popt2, pcov2 = curve_fit(f2,(galaxy_wavelengths,temp_fluxes),galaxy_fluxes,p0=[5,.1],maxfev=500000)
                    #Calculating the r**2 value for this template/scaling factor 
                    num = np.sum((galaxy_fluxes - f2((galaxy_wavelengths,temp_fluxes), *popt2))**2)
                    denom = np.sum((galaxy_fluxes - np.mean(galaxy_fluxes))**2)
                    r2_2 = 1 - num/denom
                    
                except: 
                    failed.append(i)
                
                try: 
                    #function 3 
                    popt3, pcov3 = curve_fit(f3,(galaxy_wavelengths,temp_fluxes),galaxy_fluxes,p0=[5,.1],maxfev=500000)
                    #Calculating the r**2 value for this template/scaling factor 
                    num = np.sum((galaxy_fluxes - f3((galaxy_wavelengths,temp_fluxes), *popt3))**2)
                    denom = np.sum((galaxy_fluxes - np.mean(galaxy_fluxes))**2)
                    r2_3 = 1 - num/denom
                except:
                    failed.append(i)

                #making list of each functions best r^2 value, and parameters             
                all_rs = np.array([r2,r2_2,r2_3])
                all_fs = np.array([f1,f2,f3],dtype='object')
                all_popts = np.array([popt,popt2,popt3],dtype='object')
                
                #finding the index where the best fit was 
                ind = np.where(all_rs == np.max(all_rs))[0][0]
                #taking all the information at that index 
                r2f = all_rs[ind]
                ff = all_fs[ind]
                poptf = all_popts[ind]
                
                #adding to list at very beginning outside of loop
                #so that for each time in this loop, we do this fitting for one SED 
                #out of the 30 
                r2s.append([r2f][0])
                temps.append(templatename)
                popts.append(poptf)
                fs = np.append(fs,ff)

        #Now we're looking at the best fit from each of the 30 SEDs 
        #and choosing the best one out of those 
        #to get the best overall fit and parameters 
        r2s = np.array([r2s])
        choose = np.where(r2s[0] == r2s[0].max())[0][0]
        best_temp = temps[choose]
        best_r = r2s[0][choose]
        best_f = fs[choose]
        best_popt = popts[choose]


        #Now we have our points and the SED/scaling parameters that fit best
        #Now we use that template to generate our fake points
        fake_points = [] 

        #Uf, Vf and Jf all transmission curves that will be used to make one point
        #To calculate fake photometric points
        for h in [Uf,Vf,Jf]:
            template_name = best_temp #now using best fit SED
            template = np.genfromtxt("C:/Users/19133/Documents/Research/Kirkpatrick/Templates/" + template_name)

            #Template data of best fit template

            f = best_f #function that was used  
            
            #SED wavelength values 
            temp_x2 = template[:,0] 
            
            #SED flux values 
            temp_y2 = template[:,1]*(temp_x2**2)/(3*(10**8)) #lambda to nu 
            temp_y2 = f((temp_x2,temp_y2),*best_popt) #using best function and parameters to modify SED 
            #then interpolating it to match the transmission curve 
            temp_fluxes2 = interpolate(temp_x2,temp_y2,h[:,0])  

            #Transmission curve                 
            trans_wavelengths = h[:,0]
            trans_fluxes = h[:,1]
            temp_fluxes2 = np.array(temp_fluxes2)

            #Doing calculations to generate fake point
            ydata = trans_fluxes*temp_fluxes2
            num = simps(ydata,dx=10000)
            denom = simps(trans_fluxes,dx=10000)
            fake_point = num/denom
            fake_points.append(fake_point)        

        #Writing data to file 
        final_data = np.array([[fake_points[0], fake_points[1], fake_points[2], rshift, X_soft1, X_hard1,idd]]) #Xray lum, also correct mag2flux pls
        bestfit_data = np.array([str(best_temp),str(best_r),str(best_f),str(best_popt)])
        finaldata = open("C:/Users/19133/Documents/Research/Final_Data_4_4.txt", "a+")
        np.savetxt(finaldata,final_data) 
        finaldata.close()
        bestfitdata = open("C:/Users/19133/Documents/Research/Bestfit_Data_4_4.txt", "a+")
        np.savetxt(bestfitdata,bestfit_data,fmt='%s') 
        bestfitdata.close()
        
        #un hash these to get saved set of plots 
        plt.plot(galaxy_wavelengths,galaxy_fluxes,'bo')
        plt.plot([Uang,Vang,Jang], fake_points,'ro')
        plt.plot(temp_x2,temp_y2)
        plt.xlim(10**3,10**4.5)
        plt.title(i + best_r)
        plt.loglog()
        plt.savefig("C:/Users/19133/Documents/Research/Kirkpatrick/Temp_Fits_Final/" + str(idd))
        plt.clf()

        i += 1
    except:
        failed.append(i)
        i+=1

C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  

C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in power
  # Remove the CWD from sys.path while we load stuff.
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated

C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  

C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  

C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in power
  # Remove the CWD from sys.path while we load stuff.
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated

...

C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  

C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  

C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  

C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in power
  # Remove the CWD from sys.path while we load stuff.
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated

C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Users\19133\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
